In [2]:
pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [3]:
import networkx as nx
from networkx.algorithms.community import kernighan_lin_bisection
import random
import matplotlib.pyplot as plt
from numpy import block
import scipy as sp
from sklearn.cluster import KMeans
from networkx.algorithms import community
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
from sklearn.model_selection import train_test_split
import sklearn.metrics as mt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import math


# Create a graph that can be divided into two equal-sized sets.
def dividable(numberOfNodes):
  # Create an empty graph
  G = nx.Graph()

  # Create two sets of nodes with equal size
  nodes_set1 = range(1, int(numberOfNodes) + 1)
  nodes_set2 = range(int(numberOfNodes) + 1, 2 * int(numberOfNodes) + 1)

  # Add nodes to the graph
  G.add_nodes_from(nodes_set1)
  G.add_nodes_from(nodes_set2)

  # Connect nodes from the two sets to create edges
  for node1 in nodes_set1:
      for node2 in nodes_set2:
          G.add_edge(node1, node2)

  # Now, G is a graph that can be divided into two equal-sized sets.
  return G

#Traditional Features
def traditional(G):
  features=[]
  # 1. Number of Vertices:
  num_nodes = G.number_of_nodes()

  # 2. Number of Edges:
  num_edges = G.number_of_edges()

  # 3. Average Degree of a Vertice:
  avg_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()

  # 4. Density:
  density = nx.density(G)

  # 5. Average Path Length (average shortest paths):
  # Tüm düğümler arasındaki en kısa yolları hesaplayın
  all_shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
  # En kısa yolların toplam uzunluğunu ve yol sayısını hesaplayın
  total_length = 0
  total_paths = 0
  for source_node, paths in all_shortest_paths.items():
      for target_node, length in paths.items():
          if source_node != target_node:
              total_length += length
              total_paths += 1
  # Ortalama yol uzunluğunu hesaplayın
  avg_path_length = total_length / total_paths

  # 6. Average Clustering Coefficient:
  avg_clustering = nx.average_clustering(G)

  # 7. Number of Bridges:
  bridges_count = len(list(nx.bridges(G)))

  # 8. Number of Arculation Points:
  articulation_point_count=len(list(nx.articulation_points(G)))

  # 9. Average Betweenness Centrality (Araştırıcılık):
  betweenness_centrality = nx.betweenness_centrality(G)
  avg_betweenness_centrality = sum(betweenness_centrality.values()) / len(betweenness_centrality)

  # 10. Graph Diameter:
  diameter = nx.diameter(G)

  # 11. Average Cycle:
  # Çevrimleri kontrol et
  if not nx.cycle_basis(G):
      avg_cycle_length = 0
  else:
      # Çevrimleri bulun
    cycles = list(nx.cycle_basis(G))
    # Çevrimlerin uzunluklarını hesaplayın
    cycle_lengths = [len(cycle) for cycle in cycles]
    # Ortalama çevreyi hesaplayın
    avg_cycle_length = sum(cycle_lengths) / len(cycles)

  # 12. Derece (Degree):
  degree_centrality = nx.degree_centrality(G)
  degree_centrality = list(degree_centrality.values())
  degree_centrality.sort()
  degree_centrality = np.median(degree_centrality)

  # 13. Küresellik (Closeness Centrality):
  closeness_centrality = nx.closeness_centrality(G)
  closeness_centrality = list(closeness_centrality.values())
  closeness_centrality = np.median(closeness_centrality)

  # 14. Alt Graf Büyüklüğü (Size of Subgraphs):
  subgraph_sizes = [len(subgraph) for subgraph in nx.connected_components(G)]
  subgraph_sizes=subgraph_sizes[0]

  features.append(num_nodes)
  features.append(num_edges)
  features.append(avg_degree)
  features.append(density)
  features.append(avg_path_length)
  features.append(avg_clustering)
  features.append(bridges_count)
  features.append(articulation_point_count)
  features.append(avg_betweenness_centrality)
  features.append(diameter)
  features.append(avg_cycle_length)
  features.append(degree_centrality)
  features.append(closeness_centrality)
  features.append(subgraph_sizes)

  return features

#median of features of nodes
def median_of_embeddings(G,vectors_array):
    n = G.number_of_nodes()
    final_embedded= []
    temp_embedded = []
    for j in range(vectors_array.shape[1]):
        for i in range(vectors_array.shape[0]):  # Iterate over rows
            element = vectors_array[i][j]
            temp_embedded.append(element)
        temp_embedded.sort()
        median = np.median(temp_embedded)
        final_embedded.append(median)
    return final_embedded

#Finding the average of vector's column
def average_of_embeddings(G,vectors_array):
    sum = 0
    n = G.number_of_nodes()
    embedded = []
    for j in range(vectors_array.shape[1]):
        for i in range(vectors_array.shape[0]):  # Iterate over rows

            element = vectors_array[i][j]
            sum += element
        avg = sum / n
        embedded.append(avg)
    return embedded

#Feature extraction with DeepWalk
#We will choose the walk number and walk length by trial and error
def deepwalk(G,num_walks,walk_length):
    # DeepWalk için yürüyüşler oluşturun
    walks = []

    for _ in range(num_walks):
        for node in G.nodes():
            walk = [node]
            for _ in range(walk_length - 1):
                neighbors = list(G.neighbors(walk[-1]))
                if neighbors:
                    walk.append(neighbors[0])  # Rastgele bir komşuya ilerleyin
                else:
                    break
            walks.append(walk)

    # Word2Vec modelini eğitin
    model = Word2Vec(walks, vector_size=64, window=5, min_count=0, sg=1, workers=4)

    # Gömme vektörlerini alın
    embedding_vectors = [model.wv.get_vector(node) for node in G.nodes()]

    # Herhangi bir düğümün gömme vektörünü alın

    return embedding_vectors

#Feature extraction with Node2vec
def run_n2v(G, dimensions=64, walk_length=80, num_walks=10, p=1, q=1, window=10):

    mdl = Node2Vec(
        G,
        dimensions=dimensions,
        walk_length=walk_length,
        num_walks=num_walks,
        p=p,
        q=q
    )
    mdl = mdl.fit(window=window)

    # Gömme vektörlerini alın
    vectors = [mdl.wv.get_vector(str(x)) for x in G.nodes()]
    return vectors

#Label Graphs
def labelling_kernighan(G,Y_set):
    #(|V| / 2) +(|V|x0.01)<=devided graphtaki eleman sayısı <=(|V| / 2) +(|V|x0.01) if this holds then we can label the graph as true
    #dense>500: |V| ,, sparse<360: 0.4x|V|xlog(|V|)

    #iteration number!!!!!!
    #Dense graph
    if((G.number_of_edges() >= math.comb(G.number_of_nodes(),2)/2) & (G.number_of_edges() <= math.comb(G.number_of_nodes(),2))):
      iteration_number=G.number_of_nodes()
    else:
    #sparse graph
      iteration_number=0.04*G.number_of_nodes()*math.log(G.number_of_nodes())

    partitions = community.kernighan_lin.kernighan_lin_bisection(G, max_iter=int(iteration_number))
    is_equal_partition = (len(list(partitions[0])) > (G.number_of_nodes()/2)-G.number_of_nodes()*0.01) & (len(list(partitions[0])) < (G.number_of_nodes()/2) + G.number_of_nodes() * 0.01) & (len(list(partitions[1])) > (G.number_of_nodes()/2) - G.number_of_nodes()*0.01) & (len(list(partitions[1])) < (G.number_of_nodes()/2) + G.number_of_nodes()*0.01)
    partition1 = list(partitions[0])
    partition2 = list(partitions[1])

    graph_1 = G.subgraph(partitions[0])
    graph_2 = G.subgraph(partitions[1])

    is_connected_partition1 = nx.is_connected(graph_1)
    is_connected_partition2 = nx.is_connected(graph_2)

    if is_connected_partition1 and is_connected_partition2 and is_equal_partition:
        result = True
        Y_set.append(1)
    else:
        result = False
        Y_set.append(0)



#first

#Generating Graph Data and does other calculations
def generateGraph_n2v(num_graphs, min_nodes, max_nodes):
  Y_set=[]
  embedded_list = np.zeros((num_graphs, 64))
  graph_arr = []
  i=0
  for _ in range(num_graphs):
    # Rastgele bir sayı oluşturun ve bu sayıya göre grafik türünü seçin
    graph_type = random.randint(0, 6)
    num_nodes = random.randint(min_nodes, max_nodes)
    avg_degree = random.uniform(2, 5)
    prob = random.uniform(0.1, 0.5)
    if graph_type == 0:
        # Erdos-Renyi Grafik
        G = nx.erdos_renyi_graph(n = num_nodes, p = avg_degree / num_nodes)
    elif graph_type == 1:
        # Barabasi-Albert Grafik
        m = random.randint(1,4)
        G = nx.barabasi_albert_graph(n = num_nodes, m = m)
    elif graph_type == 2:
        # Watts-Strogatz Grafik
        k = random.randint(2,6)
        G = nx.watts_strogatz_graph(n=num_nodes, k=k, p=prob)
    elif graph_type == 3:
        # Random Regular Grafik
        G = nx.random_regular_graph(d = 4, n = num_nodes)
    elif graph_type == 4:
        # Geometric Grafik
        radius = random.uniform(0.2, 0.6)
        G = nx.random_geometric_graph(n = num_nodes, radius=radius)
    elif graph_type == 5:
        # Create a random connected graph by generating a random spanning tree
        G = nx.random_tree(num_nodes)
    else:
        G=dividable(num_nodes/2)
        #print(graph_type)


    while not nx.is_connected(G):
        node1 = random.choice(list(G.nodes()))
        node2 = random.choice(list(G.nodes()))
        if node1 != node2 and not G.has_edge(node1, node2):
            G.add_edge(node1, node2)
    graph_arr.append(G)

    labelling_kernighan(G,Y_set)#labelling part

    #getting vectors of Node2vec algorithm
    vectors_array_node2vec = np.array(run_n2v(G))

    median_Node2vec_vector = median_of_embeddings(G,vectors_array_node2vec)

    embedded_list[i] = median_Node2vec_vector
    i=i+1

  return embedded_list, Y_set


In [4]:
X, Y = generateGraph_n2v(2, 400, 800)

Computing transition probabilities:   0%|          | 0/760 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:26<00:00,  2.70s/it]


Computing transition probabilities:   0%|          | 0/684 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:01<00:00,  5.19it/s]


In [5]:
import pandas as pd
def excel_olustur(X, Y):
    # X dizisini bir NumPy dizisine dönüştürün
    X = np.array(X)

    # X dizisini bir DataFrame'e dönüştürün (isteğe bağlı)
    df = pd.DataFrame(X)
    # Y vektörünü yeni bir sütun olarak DataFrame'e ekleyin
    df['label'] = Y

    # DataFrame'i Excel dosyasına kaydedebilirsiniz (isteğe bağlı)
    dosya_adi= 'node2vec_dataset_1_10k_11k.xlsx'  # Kaydetmek istediğiniz Excel dosyasının adını buraya yazın  !!!!!!!!!! ismi uygun ayarlayıp kaydetttt!!!!!!
    df.to_excel(dosya_adi, index=False)
    print("excel dosyasi olusturuldu.")


excel_olustur(X, Y)

excel dosyasi olusturuldu.
